In [1]:
import pandas as pd
import os, re
from pathlib import Path
import numpy
import requests
import slugify
from dateutil.parser import *
import numpy as np

In [2]:
prefix = """@prefix  rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix  rdfs: <http://www.w3.org/2000/01/rdf-schema#>.
@prefix  owl: <http://www.w3.org/2002/07/owl#>.
@prefix  xsd: <http://www.w3.org/2001/XMLSchema#>.
@prefix  foaf: <http://xmlns.com/foaf/0.1/>.
@prefix  halalv: <http://halal.addi.is.its.ac.id/halalv.ttl#>.
@prefix  halalf: <http://halal.addi.is.its.ac.id/foodproducts/>.
@prefix  halali: <http://halal.addi.is.its.ac.id/ingredients/>.
@prefix  halalc: <http://halal.addi.is.its.ac.id/certificates/>.
@prefix  halals: <http://halal.addi.is.its.ac.id/sources/>.
@prefix  halalm: <http://halal.addi.is.its.ac.id/manufactures/>.
@prefix  food: <http://purl.org/foodontology#> .
@prefix  foodlirmm: <http://data.lirmm.fr/ontologies/food#> .
@prefix  gr: <http://purl.org/goodrelations/v1#> . 
@prefix  dbr: <http://dbpedia.org/resource/> .

"""

In [3]:
def formatDateX(string):
    try:
        date = parse(string)
        date = date.strftime('%Y-%m-%d')
        return str(date)
    except:
        return "0000-00-00"

In [4]:
def makeSlug(string):
    slug = slugify.slugify(string)
    slug = slug.replace("-", " ")
    slug = slug.title()
    slug = slug.replace(" ", "_")
    return slug

In [5]:
def getManufactureName(string):
    print("Value string manufacture: " + str(string))
    try:
        product = str(string).split(",", 1)[0].title()
    except:
        product = str(string).title()
    print("Value now string manufacture: " + product)
    return product

In [6]:
def getProductName(string):
    print("Value string product: " + str(string))
    try:
        product = str(string).split(",", 1)[1].title()
    except:
        product = str(string).title()
    print("Value now string product: " + product)
    return product

In [7]:
def getIngredients(string):
    ingredients = str(string).title()
    return ingredients

In [8]:
def getCertTrim(string):
    cert = str(string).strip()
    return cert

In [9]:
def getTrimAndTitle(string):
    newstring = str(string).strip()
    newstring = newstring.title()
    return newstring

In [10]:
def getUniqueIngredients(list):
    ingredientList = []
    for data in list:
        #print("Data: "+data)
        for ingredient in str(data).split(","):
            #print("Get ingredient: "+ingredient)
            ingredient = ingredient.strip()
            ingredientList.append(ingredient)
    #newingredientList = list(dict.fromkeys(ingredientList))
    return ingredientList

In [11]:
def createIngredientTTL(ingredients, path):

    noOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}" .

   """

    withOwlSameAs = """halali:{slug} a foodlirmm:Ingredient;
   rdfs:label "{name}";
   owl:sameAs dbr:{slug} .

   """
    for ingredient in ingredients:
        print("Creating ingredient: " + str(ingredient))
        slugIngredient = slugify.slugify(ingredient)
        slugIngredient = str(slugIngredient).replace("-", " ")
        slugIngredient = slugIngredient.title()
        slugIngredient = slugIngredient.replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","")
    # url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F%7Bslug%7D%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+%22.format(  
    #     slug=slugIngredient)
    # r = requests.get(url)
    # data = r.text
    # if "true" in data:
    #     print("Get owl:sameAs")
    #     withOwlSameAs = withOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAs)
    #     f.close()
    #     withOwlSameAsPrefix = prefix + withOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "a+", encoding="utf-8")
    #     f.write(withOwlSameAsPrefix)
    #     f.close()
    # else:
    #     print("Doesn't get owl:sameAs")
    #     noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    #     f = open("data\\dat\\new\\jadi\\noprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAs)
    #     f.close()
    #     noOwlSameAsPrefix = prefix + noOwlSameAs
    #     f = open("data\\dat\\new\\jadi\\withprefix\\ingredients\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    #     f.write(noOwlSameAsPrefix)
    #     f.close()

        pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
        pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

        if not os.path.isdir(pathFolderNoPrefix):
            os.makedirs(pathFolderNoPrefix)

        if not os.path.isdir(pathFolderWithPrefix):
            os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
        f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAs)
        f.close()
        noOwlSameAsPrefix = prefix + noOwlSameAs
        f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
        f.write(noOwlSameAsPrefix)
        f.close()

In [12]:
def createIngredientSeparate(ingredient, path):
    ingText = """halali:{sluging} a foodlirmm:Ingredient;
   rdfs:label "{nameing}" .

   """
    
    ingredient = str(ingredient).replace('"','').replace("?","").replace("$","").replace("(e)","").replace("(","").replace(")","").replace("%","").replace(".","").replace("\\","").replace("&","")
    print("Creating ingredient: " + str(ingredient))
    slugIngredient = slugify.slugify(ingredient)
    slugIngredient = str(slugIngredient).replace("-", " ")
    slugIngredient = slugIngredient.title()
    slugIngredient = slugIngredient.replace(" ", "_").replace("%","").replace("$","").replace("(","").replace(")","")
        

    pathFolderNoPrefix = path + "\\resources\\noprefix\\ingredients"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\ingredients"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

        # print("Doesn't get owl:sameAs")
        #noOwlSameAs = noOwlSameAs.format(name=ingredient, slug=slugIngredient)
    ingText = ingText.format(sluging=slugIngredient, nameing=ingredient)
    f = open(pathFolderNoPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(ingText)
    f.close()
    noOwlSameAsPrefix = prefix + ingText
    f = open(pathFolderWithPrefix + "\\" + slugIngredient + ".ttl", "w", encoding="utf-8")
    f.write(noOwlSameAsPrefix)
    f.close()

In [13]:
def createManufactureTTL(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [14]:
def createManufactureTTLSusulan(manufacture, path):
    manufactureText = """halalm:{slug} a foaf:Organization;
		rdfs:label "{name}" .

		"""
    slugManufacture = slugify.slugify(manufacture)
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")
    manufacture = str(manufacture).replace('"','').replace("?","").replace("$","").replace("(","").replace(")","").replace("\\","").replace("&","")
    manufactureText = manufactureText.format(slug=slugManufacture, name=manufacture)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\manufactures"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\manufactures"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    f = open(pathFolderNoPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureText)
    f.close()
    manufactureTextPrefix = prefix + manufactureText
    f = open(pathFolderWithPrefix + "\\" + slugManufacture + ".ttl", "w", encoding="utf-8")
    f.write(manufactureTextPrefix)
    f.close()

In [15]:
def createCertificateTTL(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [16]:
def createCertificateTTLSusulan(row, path):
    certString = """halalc:{slugCertificate} a halalv:HalalCertificate;
			halalv:halalCode "{certificate}";
			halalv:halalExp "{expired}"^^xsd:date;
			halalv:OrgCert halals:{slugOrg};
			halalv:halalStatus {status} .

			"""

    orgString = """halals:{slugOrganization} a foaf:Organization;
    rdfs:label "{orgName}";
    foaf:homepage "{cert_bodyname}";
    foaf:based_near {location} .

    """

    expired = formatDateX(str(row['expired']))

    certCode = str(row['cert_code'])
    if ('0' == str(row['cert_code'])):
        certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
    elif ('nan' == str(row['cert_code'])):
        certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
    else:
        certCode = str(row['cert_code']).upper()

    certStatus = "halalv:"+str(row['status'])

    slugCertificate = slugify.slugify(certCode)
    slugCertificate = str(slugCertificate).replace("-", "_")
    slugCertificate = slugCertificate.upper()

    slugOrganization = slugify.slugify(str(row['cert_bodyname']))
    slugOrganization = str(slugOrganization).replace("-", " ")
    slugOrganization = slugOrganization.title()
    slugOrganization = str(slugOrganization).replace(" ", "_")

    certString = certString.format(slugCertificate=slugCertificate, certificate=str(row['cert_code']),
                                   expired=str(expired), slugOrg=slugOrganization, status=certStatus)
    orgString = orgString.format(slugOrganization=slugOrganization, orgName=str(row['cert_bodyname']),
                                 cert_bodyname=str(row['homepage']), location=str(row['location']))

    pathFolderNoPrefix = path + "\\resources\\noprefix\\certificates"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\certificates"
    pathFolderNoPrefixOrg = path + "\\resources\\noprefix\\halalsources"
    pathFolderWithPrefixOrg = path + "\\resources\\withprefix\\halalsources"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    if not os.path.isdir(pathFolderNoPrefixOrg):
        os.makedirs(pathFolderNoPrefixOrg)

    if not os.path.isdir(pathFolderWithPrefixOrg):
        os.makedirs(pathFolderWithPrefixOrg)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + slugCertificate
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + slugCertificate
    pathFileNoPrefixOrg = pathFolderNoPrefixOrg + "\\" + slugOrganization
    pathFileWithPrefixOrg = pathFolderWithPrefixOrg + "\\" + slugOrganization

    f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certString)
    f.close()
    certStringPrefix = prefix + certString
    f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
    f.write(certStringPrefix)
    f.close()

    f = open(pathFileNoPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgString)
    f.close()
    orgStringPrefix = prefix + orgString
    f = open(pathFileWithPrefixOrg + ".ttl", "w", encoding="utf-8")
    f.write(orgStringPrefix)
    f.close()

In [17]:
def createProductTTL(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [18]:
def createProductTTLSusulan(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .

halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} food:containsIngredient {concateIngredientsEntity} .

    halalf:{slugName} food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .

    halalf:{slugName} foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [19]:
def createProductTTLSusulanCoba(row, path):
    productString = """halalf:{slugName} a foodlirmm:FoodProduct;
	a food:Food;
	foodlirmm:code "{code}";
	rdfs:label "{name}";
	gr:hasManufacturer halalm:{slugManufacture} .
    foodlirmm:certificate halalc:{slugCertificate} .
    food:containsIngredient {concateIngredientsEntity} .
    food:ingredientsListAsText {concateIngredientsString} .

"""

    productStringWithoutCertificate = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        food:containsIngredient {concateIngredientsEntity} .
        food:ingredientsListAsText {concateIngredientsString} .

    """

    productStringNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
    	a food:Food;
    	foodlirmm:code "{code}";
    	rdfs:label "{name}";
    	gr:hasManufacturer halalm:{slugManufacture} .
        foodlirmm:certificate halalc:{slugCertificate} .

    """

    productStringWithoutCertificateNoIngredient = """halalf:{slugName} a foodlirmm:FoodProduct;
        	a food:Food;
        	foodlirmm:code "{code}";
        	rdfs:label "{name}";
        	gr:hasManufacturer halalm:{slugManufacture} .

        """

    product = str(row['name'])
    product = str(product).strip()
    product = product.replace('"', '').replace('\\','')
    productSlug = makeSlug(product)

    productCode = str(row['code'])

    if productCode == "nan":
        productCode = "0000"

    slugManufacture = slugify.slugify(str(row['manufacture']))
    slugManufacture = str(slugManufacture).replace("-", " ")
    slugManufacture = slugManufacture.title()
    slugManufacture = slugManufacture.replace(" ", "_")

    ingredientEntities = str(row['ingredients']).strip()
    prefixEntity = "halali:"
    stringingredientsEntities = ingredientEntities.replace(", ", ",halali:").replace(" ", "_").replace("?","").replace("$","").replace("(","").replace(")","").replace("-","").replace("=","").replace("'","").replace("&","")
    finalStringIngredientEntities = prefixEntity + stringingredientsEntities

    prefixString = "\""
    stringIngredients = ingredientEntities.replace(", ", "\", \"")
    finalStringIngredientStrings = prefixString + stringIngredients + prefixString

    certbody_name = str(row['cert_bodyname'])
    if certbody_name == "nan" and ingredientEntities == "nan":
        productString = productStringWithoutCertificateNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                                               slugManufacture=slugManufacture)
    elif certbody_name != "nan" and ingredientEntities != "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productString.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate,
                                             concateIngredientsEntity=finalStringIngredientEntities,
                                             concateIngredientsString=finalStringIngredientStrings)
    elif certbody_name != 'nan' and ingredientEntities == "nan":
        certCode = str(row['cert_code'])
        if ('0' == str(row['cert_code'])):
            certCode = str(row['expired']) + "_" + str(row['cert_bodies']).upper()
        elif ('nan' == str(row['cert_code'])):
            certCode = str(row['cert_bodyname']).upper()+"_"+str(row['status']).upper()
        else:
            certCode = str(row['cert_code']).upper()
        slugCertificate = slugify.slugify(certCode)
        slugCertificate = str(slugCertificate).replace("-", "_")
        slugCertificate = slugCertificate.upper()

        productString = productStringNoIngredient.format(slugName=productSlug, code=productCode, name=product,
                                             slugManufacture=slugManufacture,
                                             slugCertificate=slugCertificate)
    elif certbody_name == 'nan' and ingredientEntities != "nan":
        productString = productStringWithoutCertificate.format(slugName=productSlug, code=productCode,
                                                                           name=product,
                                                                           slugManufacture=slugManufacture,
                                                                           concateIngredientsEntity=finalStringIngredientEntities,
                                                                           concateIngredientsString=finalStringIngredientStrings)

    pathFolderNoPrefix = path + "\\resources\\noprefix\\foodproducts"
    pathFolderWithPrefix = path + "\\resources\\withprefix\\foodproducts"

    if not os.path.isdir(pathFolderNoPrefix):
        os.makedirs(pathFolderNoPrefix)

    if not os.path.isdir(pathFolderWithPrefix):
        os.makedirs(pathFolderWithPrefix)

    pathFileNoPrefix = pathFolderNoPrefix + "\\" + productSlug
    pathFileWithPrefix = pathFolderWithPrefix + "\\" + productSlug
    if not os.path.exists(pathFileNoPrefix):
        f = open(pathFileNoPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()
    else:
        f = open(pathFileNoPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productString)
        f.close()

    if not os.path.exists(pathFileWithPrefix):
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()
    else:
        productStringPrefix = prefix + productString
        f = open(pathFileWithPrefix + "_" + slugManufacture + ".ttl", "w", encoding="utf-8")
        f.write(productStringPrefix)
        f.close()

In [20]:
def execCreateProductTTL(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTL(row, path)

In [21]:
def execCreateProductTTLSusulan(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulan(row, path)

In [22]:
def execCreateProductTTLSusulanCoba(df, path):
    for index, row in df.iterrows():
        print("Creating product ttl on: " + str(row['name']))
        if str(row['name']) != "nan":
            createProductTTLSusulanCoba(row, path)

In [23]:
def countUniqueValuesEachEntity(df):
    print("Unique values: " + str(len(df['name'].unique().tolist())))

    # Select all duplicate rows based on one column
    duplicateRowsDF = df[df.duplicated(['name'])]
    duplicateRowsDF = duplicateRowsDF.sort_values(by=['name'])
    print("Duplicate rows length: " + str(len(duplicateRowsDF)))
    for index, row in duplicateRowsDF.iterrows():
        print("Values are: " + str(row['code']) + " - " + str(row['name']) + " - " + str(
            row['manufacture']) + " - " + str(row['cert_code']) + " - " + str(row['expired']) + " - " + str(
            row['cert_bodies']) + " - " + str(row['cert_bodyname']) + " - " + str(row['homepage']) + " - " + str(
            row['location']) + " - " + str(row['ingredients']))

In [24]:
def execCreateCertAndOrganizationTTL(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        if 'nan' == str(row['cert_bodies']):
            print("Skipping, no cert bodyname..")
            pass
        else:
            print("Creating certificate on: " + str(row['cert_code']) + str(row['cert_bodies']))
            createCertificateTTL(row, path)

In [25]:
def execCreateCertAndOrganizationTTLSusulan(df, path):
    dfcert = df[['cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'status']].copy()
    # dfcert = dfcert.dropna()
    # #
    # # df = df.drop_duplicates()
    #
    # for name in dfcert.cert_bodyname.unique():
    #     print("Values is: "+str(name))
    for index, row in dfcert.iterrows():
        #if 'nan' == str(row['cert_bodies']):
            #print("Skipping, no cert bodyname..")
            #pass
        #else:
            print("Creating certificate on: " + str(row['cert_code']) )##+ str(row['cert_bodies']))
            createCertificateTTLSusulan(row, path)

In [26]:
def execCreateIngredientsTTLFromFile(df, path):
    with open('ingredients.txt', 'r') as f:
        lines = f.readlines()
        for ingredient in lines:
            ingredient =  ingredient.replace("\n","")
            print("Working on: "+ingredient)
            createIngredientTTL(ingredient, path)

In [27]:
def execCreateIngredientsTTLFromDF(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    createIngredientTTL(ingredients, path)

In [28]:
def execCreateIngredientsTTLFromDFCoba(df, path):
    # create ingredients
    listIngredient = df['ingredients'].dropna().tolist()
    ingredients = getUniqueIngredients(listIngredient)
    newlistIng = list(dict.fromkeys(ingredients))
    for ingredient in newlistIng:
        createIngredientSeparate(ingredient, path)

In [29]:
def execCreateManufacturerTTL(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTL(manufacture, path)

In [30]:
def execCreateManufacturerTTLSusulan(df, path):
    df = df.sort_values(by ='manufacture' )
    dfObj = df['manufacture']
    for manufacture in dfObj.unique().tolist():

        if (str(manufacture) == 'nan'):
            pass
        else:
            print("Working on: " + str(manufacture))
            createManufactureTTLSusulan(manufacture, path)

In [31]:
# url = "https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=ASK+%7B%0D%0A+%3Chttp%3A%2F%2Fdbpedia.org%2Fresource%2F{slug}%3E+%3Fp+%3Fo+.%0D%0A%7D&format=text%2Fhtml&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+".format(
#     slug="Monosodium_glutamate")
# r = requests.get(url)
# data = r.text
# print("The content is: "+data)
# if "true" in data:
#     print("Get owl:sameAs")
# else:
#     print("Doesn't get owl:sameAs")

# slugIngredient = slugify.slugify("aku apa")
# print(slugIngredient)

#     createIngredientTTL()
# df['manufacture'] = df['manufacture'].apply(lambda x: getTrimAndTitle(x))
# df['name'] = df['name'].apply(lambda x: getTrimAndTitle(x))
# dforgs = df[df['cert_bodyname'] == numpy.nan]
# for dforg in dforgs:
#     print("Value of nan: "+str(len(dforg)))
# print("Size before drop mui: " + str(len(df)) + " - size unique org cert: " + str(len(df.cert_bodyname.unique())))
# df = df[~df['cert_bodyname'].isin(['https://www.halalmui.org/'])]
# print("Size after: " + str(len(df)) + " - size unique omanufacture: "+str(len(df.manufacture.unique()))+" - name "+str(len(df.name.unique())))
# for sert in df.manufacture.unique():
#     print("Sert: "+str(sert))

# for index, row in df.iterrows():
#     print("Value is: name: "+str(row['name'])+" - manufacture: "+str(row['manufacture']))

# df = pd.read_csv(path+".csv", delimiter=";", error_bad_lines=False,names=['id', 'code', 'name', 'verify', 'ingredients'])
# newdf = df[['code']].copy()
#
# newdf['name'] = df['name'].apply(lambda x: getProductName(x))
# newdf['manufacture'] = df['name'].apply(lambda x: getManufactureName(x))
# print("Length code: "+str(len(newdf['code']))+" - name: "+str(len(newdf['name']))+" - manufacture: "+str(len(newdf['manufacture'])))
# newdf['ingredientsx'] = ''
# newdf['ingredients1'] = ''
# newdf['ingredients2'] = ''
# newdf['ingredients3'] = ''
# newdf['ingredients4'] = ''
# newdf['ingredients5'] = ''
# newdf['ingredients'] = df['ingredients'].apply(lambda x: getIngredients(x))
# for data in newdf['name']:
#     print(str(data))
# for index, row in df.iterrows():
#     # print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
#     print(str(row['code'])+" - "+str(row['name'])+" - "+str(row['ingredients']))
# df['ingredients'] = ''
# df['ingredients1'] = ''
# df['ingredients2'] = ''
# df['ingredients3'] = ''
# df['ingredients4'] = ''
# df['ingredients5'] = ''
# df.to_csv(path+"_ready.csv", sep=";", index=None, header=None)

In [36]:
path = "datanew"
filename = "Hasil Salah 3"
df = pd.read_csv(path + "\\" + filename + ".csv", delimiter=",", error_bad_lines=False, low_memory=False)
df
# ,names=['code', 'name', 'manufacture', 'cert_code', 'expired', 'cert_bodies', 'cert_bodyname', 'homepage', 'location', 'ingredients', 'status']
                     

,code,name,produkmui,manufacture,cert_code,nilai,label,ingredients,ing0,ing1,...,ing29,ing30,ing31,ing32,cert_bodies,expired,cert_bodyname,homepage,location,status
0,19,Qtela Keripik Singkong Balado,Keripik singkong balado ante,Keripik ante,02120012290618,6,0,"singkong,minyak kelapa sawit,antioksidan tbhq,...",singkong,minyak kelapasawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
1,20,Qtela Keripik Singkong Balado,Keripik singkong balado ante,Keripik ante,02120012290618,6,0,"singkong,minyak kelapa sawit,antioksidan tbhq,...",singkong,minyak kelapa sawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
2,21,Qtela Keripik Tempe Original,Keripik tempe original,Keripik tempe tempelan,05100021541120,6,0,"tempe,minyak kelapa sawit antioksidan tbhq,tep...",tempe,minyak kelapa sawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
3,22,Kusuka Keripik Singkong Balado,Keripik singkong balado ante,Keripik ante,02120012290618,6,0,"singkong,minyak kelapa sawit,gula,bumbu rasa b...",singkong,minyak kelapa sawit,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
4,23,Karuhun Keripik Singkong Original,Kita-kita keripik singkong original,Kita-kita,09100013370720,6,0,"keripik singkong piliihan,daun jeruk,garam,cab...",keripik singkong piliihan,daun jeruk,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,1276,Pro Chiz Keju Slice 5'S,Keju spesial,Esthree,15160029640418,"1,42857E+16",0,kosong,kosong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
703,1277,Pro Chiz Keju Slice 10'S,Keju spesial,Esthree,15160029640418,"1,42857E+16",0,"water,cheddar cheese,vegetable oil,thickener a...",water,cheddar cheese,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
704,1278,Tango Drink Don Pedro,Beng beng drink,Rirora dcafe,05160022181220,"1,42857E+16",0,kosong,kosong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
705,1279,Milo Healthy Drink Polybag Activ-Go,Dr milo,Dr chicken,00160113750121,"1,42857E+16",0,"sari malt,barli,gula,susu bubuk skim,maltodeks...",sari malt barli,gula,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal


In [37]:
NaN = np.nan
df["cert_bodies"] = NaN
df["expired"] = NaN
df["cert_bodyname"] = "Majelis Ulama Indonesia"
df["homepage"] = NaN
df["location"] = NaN
df["ingredients"] = df["ingredients"].replace('kosong', np.nan , regex=True)
df["ingredients"] = df["ingredients"].replace(',', ', ' , regex=True)
##df["ingredients"] = df["ingredients"].replace('.', '' , regex=True)
df["status"] = "Halal"
df["name"] = df["name"].str.lower()
df["manufacture"] = df["manufacture"].str.lower()
df["ingredients"] = df["ingredients"].str.lower()
##df = df[df.label == 0]
##df
##df.to_csv("Hasil Salah.csv")
df

,code,name,produkmui,manufacture,cert_code,nilai,label,ingredients,ing0,ing1,...,ing29,ing30,ing31,ing32,cert_bodies,expired,cert_bodyname,homepage,location,status
0,19,qtela keripik singkong balado,Keripik singkong balado ante,keripik ante,02120012290618,6,0,"singkong, minyak kelapa sawit, antioksidan tbh...",singkong,minyak kelapasawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
1,20,qtela keripik singkong balado,Keripik singkong balado ante,keripik ante,02120012290618,6,0,"singkong, minyak kelapa sawit, antioksidan tbh...",singkong,minyak kelapa sawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
2,21,qtela keripik tempe original,Keripik tempe original,keripik tempe tempelan,05100021541120,6,0,"tempe, minyak kelapa sawit antioksidan tbhq, t...",tempe,minyak kelapa sawit antioksidan tbhq,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
3,22,kusuka keripik singkong balado,Keripik singkong balado ante,keripik ante,02120012290618,6,0,"singkong, minyak kelapa sawit, gula, bumbu ras...",singkong,minyak kelapa sawit,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
4,23,karuhun keripik singkong original,Kita-kita keripik singkong original,kita-kita,09100013370720,6,0,"keripik singkong piliihan, daun jeruk, garam, ...",keripik singkong piliihan,daun jeruk,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,1276,pro chiz keju slice 5's,Keju spesial,esthree,15160029640418,"1,42857E+16",0,NaN,kosong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
703,1277,pro chiz keju slice 10's,Keju spesial,esthree,15160029640418,"1,42857E+16",0,"water, cheddar cheese, vegetable oil, thickene...",water,cheddar cheese,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
704,1278,tango drink don pedro,Beng beng drink,rirora dcafe,05160022181220,"1,42857E+16",0,NaN,kosong,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal
705,1279,milo healthy drink polybag activ-go,Dr milo,dr chicken,00160113750121,"1,42857E+16",0,"sari malt, barli, gula, susu bubuk skim, malto...",sari malt barli,gula,...,NaN,NaN,NaN,NaN,NaN,NaN,Majelis Ulama Indonesia,NaN,NaN,Halal


In [38]:
execCreateProductTTLSusulan(df, path)
execCreateCertAndOrganizationTTLSusulan(df, path)
execCreateManufacturerTTLSusulan(df, path)
execCreateIngredientsTTLFromDF(df, path)

Creating product ttl on: qtela keripik singkong balado
Creating product ttl on: qtela keripik singkong balado
Creating product ttl on: qtela keripik tempe original
Creating product ttl on: kusuka keripik singkong balado
Creating product ttl on: karuhun keripik singkong original
Creating product ttl on: kusuka keripik singkong original
Creating product ttl on: indomaret keripik pisang madu
Creating product ttl on: indomaret snack kue tambang
Creating product ttl on: qtela keripik singkong original
Creating product ttl on: qtela keripik singkong original
Creating product ttl on: luwak kopi bubuk murni
Creating product ttl on: nescafe coffee cream
Creating product ttl on: golda coffee drink dolce latte
Creating product ttl on: nestle air mineral pure life
Creating product ttl on: x'tragin minuman jahe merah
Creating product ttl on: kayarasa jahe merah instant
Creating product ttl on: karuhun keripik singkong original sp
Creating product ttl on: lovy butter cookies choco butter
Creating pr

Creating product ttl on: top coffee kopi gula toraja
Creating product ttl on: neo coffee kopi instant moccachino
Creating product ttl on: neo coffee kopi instant caramel machiato
Creating product ttl on: supermi nutrimi mie goreng steak ayam
Creating product ttl on: 3 ayam mie telor super kuning
Creating product ttl on: pororo minuman berperisa stroberi
Creating product ttl on: kraft keju cheddar olahan
Creating product ttl on: kraft keju cheddar olahan
Creating product ttl on: cheetos snack jagung bakar
Creating product ttl on: cheetos snack jagung bakar
Creating product ttl on: mogu-moguminuman nata de coco strawberry
Creating product ttl on: panda grass jelly drink
Creating product ttl on: skippy jam creamy peanut butter
Creating product ttl on: kusuka keripik singkong keju bakar
Creating product ttl on: aqua galon kosong + isi ulang
Creating product ttl on: sukses's mie goreng isi 2 ayam kremes
Creating product ttl on: indomie mi instan soto mie
Creating product ttl on: indomie mi 

Creating product ttl on: lo han kuo minuman infusion
Creating product ttl on: lay's snack potato chips bbq fiesta
Creating product ttl on: happytos snack tortilla chips roasted corn
Creating product ttl on: milo healthy drink 3 in 1 actigen-e
Creating product ttl on: indofood sambal ala bangkok asam manis
Creating product ttl on: morinaga chil school susu pertumbuhan vanila
Creating product ttl on: happytos snack tortilla chips jagung bakar
Creating product ttl on: oishi snack kraker u,g pedas
Creating product ttl on: swiss miss dark chocolate drink
Creating product ttl on: kapal api krim kafe non dairy creamer
Creating product ttl on: aik cheong white coffee original
Creating product ttl on: khong guan wafer stick chocolate
Creating product ttl on: nabati bisvit marie sandwich cheese cream
Creating product ttl on: garuda snack potato daging sapi bbq
Creating product ttl on: alpenliebe candy chocolate caramel eclairs
Creating product ttl on: loacker wafer quadratini cappuccino
Creating

Creating product ttl on: khong guan crackers saltcheese combociz
Creating product ttl on: khong guan crackers saltcheese combociz
Creating product ttl on: kartika toast soesnack chocolate
Creating product ttl on: san carlo keripik kentang piu gusto paprika
Creating product ttl on: sie/kim teng-teng
Creating product ttl on: dilmah greean tea jasmine petals 20's envelope
Creating product ttl on: big babol candy stick blueberry
Creating product ttl on: kraft sandwich oreo chocolate creme
Creating product ttl on: chitato snack potato chips ayam bumbu
Creating product ttl on: happytos snack tortila chips hijau
Creating product ttl on: kit kat chocolate 4 fingers
Creating product ttl on: selma pumpkin seeds
Creating product ttl on: happytos snack tortila chips merah
Creating product ttl on: roma biscuit susu+selai slai o'lai blueberry
Creating product ttl on: hampton shortbread pure butter chocolate
Creating product ttl on: s-26 procal susu bubuk pertumbuhan 3 vanilla
Creating product ttl on

Creating product ttl on: biskuat biscuit wonderfull
Creating product ttl on: bakalland crunchy muesli 5 tropical fruits,honey
Creating product ttl on: bakalland crunchy muesli 5 seeds,honey
Creating product ttl on: bakalland crunchy muesli 5 forest fruits,honey
Creating product ttl on: bakalland crunchy muesli 5 nuts,honey
Creating product ttl on: aik cheong coffeemix reguler
Creating product ttl on: kayarasa jasela instant
Creating product ttl on: matchamu makjamu
Creating product ttl on: kolln muesli strawbry yogurt
Creating product ttl on: loacker wafer quadratini napolitaner
Creating product ttl on: pro chiz keju slice 5's
Creating product ttl on: pro chiz keju slice 10's
Creating product ttl on: tango drink don pedro
Creating product ttl on: milo healthy drink polybag activ-go
Creating product ttl on: maling/mailing greenpeas
Creating certificate on: 02120012290618
Creating certificate on: 02120012290618
Creating certificate on: 05100021541120
Creating certificate on: 021200122906

...

Creating ingredient: sirup jagung tinggi fruktosa
Creating ingredient: pengemulsi
Creating ingredient: lesitin kedelai
Creating ingredient: air
Creating ingredient: gula
Creating ingredient: madu
Creating ingredient: fruktosa
Creating ingredient: glukosa
Creating ingredient: konsentrat leci
Creating ingredient: perisa alami leci
Creating ingredient: pengatur keasaman
Creating ingredient: asam sitrat
Creating ingredient: kalsium laktat
Creating ingredient: pengawet kalium sorbat
Creating ingredient: kafein
Creating ingredient: tepung terigu
Creating ingredient: margarine
Creating ingredient: antioksidan bha
Creating ingredient: pewarna beta karoten ci  anato ci
Creating ingredient: kurkumin ci
Creating ingredient: lemak nabati
Creating ingredient: antioksidan bha
Creating ingredient: gula
Creating ingredient: garam
Creating ingredient: minyak nabati
Creating ingredient: gula
Creating ingredient: glukosa
Creating ingredient: kecap bubuk
Creating ingredient: garam
Creating ingredient: pen

Creating ingredient: cokelat rendah lemak
Creating ingredient: tepung oat
Creating ingredient: sirup karamel
Creating ingredient: sirup gula invert
Creating ingredient: ekstrak barley malt
Creating ingredient: garam
Creating ingredient: pengemulsi lesitin
Creating ingredient: kedelai
Creating ingredient: maltodekstrin
Creating ingredient: minyak nabati
Creating ingredient: kaseinat
Creating ingredient: dekstrin
Creating ingredient: protein whey
Creating ingredient: isomaltulosa
Creating ingredient: inulin
Creating ingredient: pemanis alami sorbitol
Creating ingredient: perisa identik alami
Creating ingredient: vanili
Creating ingredient: susu
Creating ingredient: pengemulsi lesitin kedelai
Creating ingredient: pemanis buatan sukralosa
Creating ingredient: mineral
Creating ingredient: vitamin
Creating ingredient: pati jagung terhidrolisa
Creating ingredient: natrium
Creating ingredient: kalsium kaseinat
Creating ingredient: sukrosa
Creating ingredient: minyak bunga matahari tinggi oleat

Creating ingredient: pengawet
Creating ingredient: pemanis buatan
Creating ingredient: penstabil nabati
Creating ingredient: sekuestran
Creating ingredient: premiks vitamin
Creating ingredient: air
Creating ingredient: fruktosa
Creating ingredient: pengatur keasaman
Creating ingredient: gula
Creating ingredient: perisa sintetik
Creating ingredient: ekstrak teh putih
Creating ingredient: natrium klorida
Creating ingredient: trinatrium sitrat
Creating ingredient: kalium karbonat
Creating ingredient: kalium klorida
Creating ingredient: pengawet
Creating ingredient: pemanis buatan
Creating ingredient: penstabil nabati
Creating ingredient: sekuestran
Creating ingredient: premiks vitamin
Creating ingredient: gula pasir
Creating ingredient: tepung ketan
Creating ingredient: santan kelapa
Creating ingredient: gula merah
Creating ingredient: lemak sapi
Creating ingredient: minyak sayur
Creating ingredient: tepung terigu
Creating ingredient: cokelat
Creating ingredient: susu
Creating ingredient:

Creating ingredient: ekstrak sereh
Creating ingredient: ekstrak kulit manggis
Creating ingredient: ekstrak madu
Creating ingredient: vitamin c
Creating ingredient: krimer nabati
Creating ingredient: gula
Creating ingredient: tehhijau
Creating ingredient: tepung terigu
Creating ingredient: minyak nabati
Creating ingredient: gula
Creating ingredient: glukosa
Creating ingredient: fruktosa
Creating ingredient: gliserin
Creating ingredient: puree apel
Creating ingredient: susu bubuk
Creating ingredient: garam
Creating ingredient: pengemulsi lesitin kedelai
Creating ingredient: pengatur keasaman
Creating ingredient: penstabil nabati
Creating ingredient: pengembang
Creating ingredient: kalsium karbonat
Creating ingredient: konsentrat stroberi
Creating ingredient: perisa sintetik
Creating ingredient: pewarna makanan
Creating ingredient: premiks vitamin
Creating ingredient: tepung terigu
Creating ingredient: mentega
Creating ingredient: gula
Creating ingredient: telur
Creating ingredient: kelap

Creating ingredient: penstabil dinatrium difosfat
Creating ingredient: vanili
Creating ingredient: kacang hazel
Creating ingredient: pengemulsi lesitin kedelai
Creating ingredient: rempah rempah
Creating ingredient: kayu manis
Creating ingredient: water
Creating ingredient: cheddar cheese
Creating ingredient: vegetable oil
Creating ingredient: thickener acid treated starch
Creating ingredient: emulsifiers
Creating ingredient: sodium citrate
Creating ingredient: disodium phosphate
Creating ingredient: sodium heksamethaphosphate
Creating ingredient: garam
Creating ingredient: acidity regulator lactic acid
Creating ingredient: natural cheese flavour
Creating ingredient: preservatives
Creating ingredient: potassium sorbate
Creating ingredient: nisin
Creating ingredient: coloring annatto cl
Creating ingredient: sari malt
Creating ingredient: barli
Creating ingredient: gula
Creating ingredient: susu bubuk skim
Creating ingredient: maltodekstrin
Creating ingredient: bubuk kakao
Creating ingre

In [39]:
execCreateIngredientsTTLFromDFCoba(df, path)

Creating ingredient: singkong
Creating ingredient: minyak kelapa sawit
Creating ingredient: antioksidan tbhq
Creating ingredient: amonium bikarbonat
Creating ingredient: bumbu rasa balado
Creating ingredient: mononatrium glutamat
Creating ingredient: ponceau r ci
Creating ingredient: kuning fcf ci
Creating ingredient: penguat rasa mononatrium glutamat
Creating ingredient: pewarna makanan
Creating ingredient: ponceau rc ci- kuning fcf ci
Creating ingredient: tempe
Creating ingredient: minyak kelapa sawit antioksidan tbhq
Creating ingredient: tepung beras
Creating ingredient: tapioka
Creating ingredient: campuran bumbu
Creating ingredient: dinatrium inosinat
Creating ingredient: dinatrium guanilat
Creating ingredient: asam askorbat
Creating ingredient: tepung telur
Creating ingredient: gula
Creating ingredient: garam
Creating ingredient: keripik singkong piliihan
Creating ingredient: daun jeruk
Creating ingredient: cabai
Creating ingredient: minyak goreng
Creating ingredient: rempah remp

Creating ingredient: jeruk nipis
Creating ingredient: bawang
Creating ingredient: asesulfam-k ppm
Creating ingredient: kakao bubuk
Creating ingredient: penstabil
Creating ingredient: monohidrat
Creating ingredient: cokelat bubuk
Creating ingredient: ekstrak malt
Creating ingredient: perisa identik alami vanili
Creating ingredient: perisa alami susu
Creating ingredient: perisa identik alami cokelat italia
Creating ingredient: jahe
Creating ingredient: lemak nabati
Creating ingredient: biji bunga matahari
Creating ingredient: perisa artifisial rempah - rempah rempah
Creating ingredient: perisa artifisial teh hijau
Creating ingredient: serpihan kentang
Creating ingredient: pewarna alami kurkumin ci
Creating ingredient: megandung ekstrak sapi
Creating ingredient: mentega
Creating ingredient: kelapa bubuk
Creating ingredient: sirup glukosa
Creating ingredient: kismis
Creating ingredient: sari buah
Creating ingredient: mangga
Creating ingredient: pir
Creating ingredient: perisa sintetik mang

Creating ingredient: keragenan
Creating ingredient: konnyaku bubuk
Creating ingredient: asam
Creating ingredient: kalium sitrat
Creating ingredient: serat pangan frukto oligasakarida
Creating ingredient: perisa melon
Creating ingredient: biru berlian ci
Creating ingredient: perisa stroberi
Creating ingredient: pewarna makanan karmoisin ci
Creating ingredient: pewarna alami karamel e c
Creating ingredient: d
Creating ingredient: b
Creating ingredient: buah jambu
Creating ingredient: perisa jambu
Creating ingredient: vitamin c
Creating ingredient: pewarna alami karmin
Creating ingredient: pemantap nabati
Creating ingredient: pengental
Creating ingredient: gom arab
Creating ingredient: gom gellant
Creating ingredient: mint
Creating ingredient: ester sukrosa
Creating ingredient: asam lemak
Creating ingredient: pelapis
Creating ingredient: karnauba wax
Creating ingredient: sari kacang hijau
Creating ingredient: premiks mineral
Creating ingredient: daun pan
Creating ingredient: saos sambal
C

Creating ingredient: gulam cokelat pasta
Creating ingredient: susu bubuk lemak nabati
Creating ingredient: pengental gum arab
Creating ingredient: tepung kedelai
Creating ingredient: ekstrak barley malt
Creating ingredient: cokelat rendah lemak
Creating ingredient: penstabil dinatrium difosfat
Creating ingredient: kacang hazel
Creating ingredient: kayu manis
Creating ingredient: glucose syrup
Creating ingredient: hazelnuts  in the cream
Creating ingredient: sweet whey powder
Creating ingredient: soy flour
Creating ingredient: skimmed milk powder
Creating ingredient: fat reduced cocoa
Creating ingredient: malt extract
Creating ingredient: raising agents
Creating ingredient: sodium hydrogen carbonate
Creating ingredient: disodium difosfat
Creating ingredient: emulsifier soy lecithins
Creating ingredient: vanili pods
Creating ingredient: bourbon pure coconut oil
Creating ingredient: fine chocolate
Creating ingredient: specially selected hazelnuts from naples and authentic vanili pods
Crea

Creating ingredient: perisa keju bubuk
Creating ingredient: pengatur asam sitrat
Creating ingredient: perisa siantetik keju
Creating ingredient: perisa artifisial cocopan
Creating ingredient: sari pan
Creating ingredient: rolled oats
Creating ingredient: biji perila
Creating ingredient: minyak beras
Creating ingredient: isomalto-oligosakarida
Creating ingredient: anti kempal
Creating ingredient: oat
Creating ingredient: ektrak malt
Creating ingredient: serealia
Creating ingredient: tepung gandum utuh
Creating ingredient: sari malt
Creating ingredient: pengemulsi lesitin kedelai perisa
Creating ingredient: antioksidan tokoferol nestle
Creating ingredient: bumbu rasa sambal balado
Creating ingredient: bumbu rasa ayam bakar
Creating ingredient: bubuk ayam
Creating ingredient: gochujang
Creating ingredient: cabai giling kasar
Creating ingredient: kecap asin
Creating ingredient: tahu kering
Creating ingredient: fish cake
Creating ingredient: perisa sintetik tomat
Creating ingredient: pengat

Creating ingredient: pewarna annatto
Creating ingredient: min
Creating ingredient: db
Creating ingredient: trikalsium fosfat
Creating ingredient: pewarna alaanato ci
Creating ingredient: pewarna beta karoten ci beta-karoten
Creating ingredient: natrium sakasin
Creating ingredient: pewarna beta karoten
Creating ingredient: okra
Creating ingredient: ikan mackerel
Creating ingredient: saos tomat
Creating ingredient: merah
Creating ingredient: ekstrak krisantemum
Creating ingredient: ekstrak bengkuang
Creating ingredient: pemanis buatan asesulfame
Creating ingredient: oat serpihan
Creating ingredient: keripik pisang
Creating ingredient: beras ekstrudat
Creating ingredient: gandum serpihan
Creating ingredient: manisan nanas
Creating ingredient: manisan pepaya
Creating ingredient: kelapa kering
Creating ingredient: pengemulsi lesitin nabati
Creating ingredient: biji rami
Creating ingredient: biji wijen
Creating ingredient: biji amaranthus
Creating ingredient: manisan cranberry
Creating ingre